In [1]:
import pickle
import numpy as np
import pandas as pd

## Load Data

In [2]:
path_subscribers = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Marketing Analytics\\Final Project\\Fall 2021 MSDABC Final Project\\Digital Marketing Analytics final case project data\subscribers"
path_engagement = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Marketing Analytics\\Final Project\\Fall 2021 MSDABC Final Project\\Digital Marketing Analytics final case project data\engagement"
path_reps = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Marketing Analytics\\Final Project\\Fall 2021 MSDABC Final Project\\Digital Marketing Analytics final case project data\customer_service_reps"

In [3]:
# Build a function that loads the binary data

def load_data(path):
    
    '''
    The input of the function is the path of the data file.
    The output of the function is the dataframe loaded.
    '''
    
    f=open(path,"rb")
    df=pickle.load(f)
    f.close()
    
    return df

In [4]:
df_subs = load_data(path_subscribers)
df_reps = load_data(path_reps)

In [5]:
# Read the channel data

df_channels = pd.read_excel("advertising_spend_data.xlsx", header=2)

## Data Review

First, we check the dataset to see which data we want to use for A/B testing and Allocation method.

In [6]:
rep_customers = df_reps['subid'].unique()
print("In representative dataset, there are records for {} past or present customers.".format(len(rep_customers)))

In representative dataset, there are records for 1369360 past or present customers.


In [7]:
sub_customers = df_subs['subid'].unique()
print("In subsriber dataset, there are records for {} past or present customers.".format(len(sub_customers)))

In subsriber dataset, there are records for 227628 past or present customers.


We're not sure why there are far more customer records in the representative dataset. But merging them might not be a reasonable approach since subids in the subscriber dataset is not a subset of those in the representative dataset.

Let's check the less complex representative dataset first.

We can see that trial plan are for each customer is avialable in this dataset, meaning that representative data might be ideal for our A/B testing.

In [8]:
df_reps['num_trial_days'].value_counts()

14    1968089
7      160596
0       79958
Name: num_trial_days, dtype: int64

Before we start performing A/B testings, we need to find our what are the variants worth testing.<br>
We noticed that the distribution of billing channel is somewhat similar to the distribution of trial plans. <br>
We want to see if it's the case that certain trial plans were only available to certain channels.<br>

In [9]:
# Examine distribution of billing channels

df_reps['billing_channel'].value_counts()

OTT       1848663
itunes     301713
google      58267
Name: billing_channel, dtype: int64

In [10]:
# Examine billing channels with no trial period plans.

df_reps[df_reps['num_trial_days'] == 0]['billing_channel'].value_counts()

google    58267
OTT       21688
itunes        3
Name: billing_channel, dtype: int64

In [11]:
# Examine billing channels with 7-day trial period plans.

df_reps[df_reps['num_trial_days'] == 7]['billing_channel'].value_counts()

itunes    160596
Name: billing_channel, dtype: int64

In [12]:
# Examine billing channels with 14-day trial period plans.

df_reps[df_reps['num_trial_days'] == 14]['billing_channel'].value_counts()

OTT       1826975
itunes     141114
Name: billing_channel, dtype: int64

We can see that what itunes offered were mostly **7-day trials** and **14-day trials**.<br>
Google offered **no trial** period to customers. <br>
OTT offered **14-day trials** and **no trail** plans.


This indicates that we might need to segment customer population when doing the A/B test.<br>
Given the data we had, there are two things we can test: <br>
**14-Day Trial** vs. **7-Day Trial** for itunes customers.<br>
**14-Day Trial** vs. **No Trial** for OTT customers.<br>
<br>
Further hypotheses cannot be tested given the data we had. But in future analysis, we might be also to make some valuable inferences by integrating our two test results with some Diff-in-Diff comparison.

Next, let's check the subscriber data to see how we can extract valuable information for attribution and allocation.

In [13]:
# Build a function that get the account creation month

def get_month(datetime):
    
    '''
    The input of the function is datetime.
    The output of the function is month.
    '''
    
    year, month, date = str(datetime).split('-')
    time = '/'.join([year, month])
    
    return time

In [14]:
# Replace the original date with the recalculated date

df_channels['date'] = df_channels['date'].apply(get_month)
df_subs['account_creation_month'] = df_subs['account_creation_date'].apply(get_month)

In [15]:
df_channels

,date,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
0,2019/06,84800,75000,17300,21400,11900,10500,5900,6100
1,2019/07,60000,75600,17300,21300,12000,29800,6000,6600
2,2019/08,52300,76100,18100,21300,16300,65900,6200,6800
3,2019/09,49000,75800,18900,21300,16400,73700,6000,7400
4,2019/10,51300,76000,21000,21400,12200,32300,6700,7300
5,2019/11,56100,75900,21400,21600,11700,9800,6300,7300
6,2019/12,51400,76100,32100,22200,12100,3500,7200,8000
7,2020/01,49000,75500,33600,22700,14300,300,6700,7400
8,2020/02,49700,76600,30100,21700,15000,300,6400,6500
9,2020/03,49100,78000,22700,21200,17600,300,5900,10000


In [16]:
# Calculate the budget for each month

df_channels.iloc[:,1:].apply(sum, axis = 1)

0    232900
1    228600
2    263000
3    268500
4    228200
5    210100
6    212600
7    209500
8    206300
9    204800
dtype: int64

In [17]:
# Check the total number of customers converted every month

df_subs['account_creation_month'].value_counts()

2020/02    28089
2019/11    26828
2020/03    26122
2019/07    25708
2019/08    25434
2019/12    25339
2020/01    24843
2019/10    22412
2019/09    20190
2019/06     2663
Name: account_creation_month, dtype: int64

We can confirm that, to calculate attribution and allocation, both subscriber and channel data should be used.<br>
Next, let's think about which data implies conversion.

In [18]:
channels= list(df_channels.columns[1:])
channels

['facebook',
 'email',
 'search',
 'brand sem intent google',
 'affiliate',
 'email_blast',
 'pinterest',
 'referral']

In [19]:
df_paid = df_subs[df_subs['paid_TF'] == True]
df_convert = df_paid[df_paid['refund_after_trial_TF'] == False]

We find it reasonable to consider customers who have made a successful payment and haven't requested refund as converted customers.

In [20]:
df_convert.shape

(82805, 30)

In [21]:
# Check the data we have for attribution

df_convert['attribution_technical'].value_counts()[channels]

facebook                   26279
email                      10007
search                      8095
brand sem intent google     7573
affiliate                   3039
email_blast                 3589
pinterest                   1853
referral                    2797
Name: attribution_technical, dtype: int64

In [22]:
for channel in channels:
    if channel in df_convert['attribution_survey'].value_counts().index:
        print(channel, df_convert['attribution_survey'].value_counts()[channel])

facebook 41142
search 2613
affiliate 639
pinterest 2493
referral 10305


In [23]:
months = sorted(list(df_convert['account_creation_month'].unique()))

In [24]:
df_convert_bymonth = df_convert.groupby("account_creation_month")

In [25]:
df_convert_bymonth.get_group(months[0])['attribution_technical'].value_counts()[channels]

facebook                   497
email                      135
search                      35
brand sem intent google     91
affiliate                   39
email_blast                 61
pinterest                   10
referral                    15
Name: attribution_technical, dtype: int64

In [26]:
df_channels

,date,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
0,2019/06,84800,75000,17300,21400,11900,10500,5900,6100
1,2019/07,60000,75600,17300,21300,12000,29800,6000,6600
2,2019/08,52300,76100,18100,21300,16300,65900,6200,6800
3,2019/09,49000,75800,18900,21300,16400,73700,6000,7400
4,2019/10,51300,76000,21000,21400,12200,32300,6700,7300
5,2019/11,56100,75900,21400,21600,11700,9800,6300,7300
6,2019/12,51400,76100,32100,22200,12100,3500,7200,8000
7,2020/01,49000,75500,33600,22700,14300,300,6700,7400
8,2020/02,49700,76600,30100,21700,15000,300,6400,6500
9,2020/03,49100,78000,22700,21200,17600,300,5900,10000


There are multiple tasks we can do for attribution and allocation.<br>
First, for attribution method, it's obvious that we shall attribute conversion based on both ***attribution_technical*** and ***attribution_survey***.<br>
Second, we want to calculate CAC for all channels in all months to see if how CAC differs between channels and how CAC of each channel change over time. <br>
Third, using monthly data, we can calculate marginal CAC for allocation decision making. This approach should be viable since monthly budgets for most channels didn't change a lot except channel **email_blast**. This might mean that marginal CAC calculated for email_blast would not be representative.<br>